In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import matplotlib.pyplot as plt
import seaborn as sns

# EPD Capacity Log
Created using Computer-Aided Dispatch data from EPD dispatch between January 1, 2014 and December 31, 2024.

### Import Note
This log reflects the impact of *CAHOOTS'* capacity on the Eugene Police Department, **not** the EPD's own vehicle capacities..

### Import CAD data

In [2]:
cad = pd.read_csv("../data/cleaned_full_class_data.csv")

#### Convert `calltime` to a datetime object.

In [3]:
cad['calltime'] = pd.to_datetime(cad['calltime'], errors='coerce')

#### Create new time stamp variables, `dispatchtime` and `endtime`, by adding `secs_to_disp` and `secs_to_close` variables to `calltime`.

In [4]:
cad['dispatchtime'] = cad['calltime'] + pd.to_timedelta(cad['secs_to_disp'], unit='s')
cad['endtime'] = cad['calltime'] + pd.to_timedelta(cad['secs_to_close'], unit='s')

#### Separate EPD and CAHOOTS calls into *seperate* DataFrames.

In [5]:
epd_calls = cad[cad['agency'] == 'EPD'].copy()
cahoots_calls = cad[cad['agency'] == 'CAHOOTS'].copy()

### Create time stamps for each call start and end time
The DataFrame `events` has **four** variables:
- `time`: either call start or end time
- `change`: if call start, `change == 1`, if call end, `change == 0`
- `cahoots_units_busy`: number of overlapping calls where the primary responding unit is from CAHOOTS
- `scheduled_capacity`: same logic as the CAHOOTS Busy Log, the second van starts operating 1/1/2017 from 5am-12pm

In [6]:
starts = cahoots_calls[['calltime']].copy()
starts['change'] = 1
starts = starts.rename(columns={'calltime': 'time'})

ends = cahoots_calls[['endtime']].copy()
ends['change'] = -1
ends = ends.rename(columns={'endtime': 'time'})

events = pd.concat([starts, ends]).sort_values('time').reset_index(drop=True)
events['cahoots_units_busy'] = events['change'].cumsum()

In [7]:
def scheduled_capacity(time):
    hour = time.hour
    if 5 <= hour < 12:
        return 2
    else:
        return 1

events['scheduled_capacity'] = events['time'].apply(scheduled_capacity)

In [8]:
events

,time,change,cahoots_units_busy,scheduled_capacity
0,2014-01-01 01:55:39,1,1,1
1,2014-01-01 02:34:38,-1,0,1
2,2014-01-01 12:50:34,1,1,1
3,2014-01-01 13:02:58,-1,0,1
4,2014-01-01 13:29:17,1,1,1
...,...,...,...,...
250225,2024-12-31 21:11:16,1,2,1
250226,2024-12-31 21:21:56,1,3,1
250227,2024-12-31 22:02:29,-1,2,1
250228,2024-12-31 22:29:10,-1,1,1


### Merge the `events` and `epd_calls` DataFrames by `time` and `calltime`, respectively.

In [9]:
epd_calls = epd_calls.sort_values('calltime')
events = events.sort_values('time')

epd_with_capacity = pd.merge_asof(
    epd_calls,
    events[['time', 'cahoots_units_busy', 'scheduled_capacity']],
    left_on='calltime',
    right_on='time',
    direction='backward'
)

epd_with_capacity['cahoots_at_capacity'] = (epd_with_capacity['cahoots_units_busy'] >= epd_with_capacity['scheduled_capacity']).astype(int)

#### Remove rows with missing values **JUST ADDDED

In [10]:
epd_with_capacity = epd_with_capacity[epd_with_capacity['time'].notnull()]

In [11]:
epd_with_capacity

,yr,service,inci_id,calltime,case_id,callsource,nature,closecode,closed_as,secs_to_disp,...,mins_disp_to_arrv,hrs_disp_to_arrv,hours_to_disp,hours_to_arrv,dispatchtime,endtime,time,cahoots_units_busy,scheduled_capacity,cahoots_at_capacity
8,2014,LAW,14000115,2014-01-01 02:09:50,NaN,PHONE,ASSAULT,UTL,UNABLE TO LOCATE,264.0,...,3.083333,0.051389,0.073333,0.124722,2014-01-01 02:14:14,2014-01-01 02:25:52,2014-01-01 01:55:39,1.0,1.0,1
9,2014,LAW,14000120,2014-01-01 02:13:32,1400012.0,E911,DISPUTE,REPT,REPORT TAKEN,188.0,...,5.783333,0.096389,0.052222,0.148611,2014-01-01 02:16:40,2014-01-01 08:54:05,2014-01-01 01:55:39,1.0,1.0,1
10,2014,LAW,14000124,2014-01-01 02:26:18,NaN,W911,DISORDERLY SUBJECT,RSLV,RESOLVED,172.0,...,3.966667,0.066111,0.047778,0.113889,2014-01-01 02:29:10,2014-01-01 03:08:03,2014-01-01 01:55:39,1.0,1.0,1
11,2014,LAW,14000149,2014-01-01 02:59:20,NaN,PHONE,ASSAULT,UNFD,UNFOUNDED,21133.0,...,13.066667,0.217778,5.870278,6.088056,2014-01-01 08:51:33,2014-01-01 09:30:41,2014-01-01 02:34:38,0.0,1.0,0
12,2014,LAW,14000161,2014-01-01 03:18:23,1400011.0,E911,VEHICLE/PEDESTRIAN CRASH,ARR,ARREST,14.0,...,1.183333,0.019722,0.003889,0.023611,2014-01-01 03:18:37,2014-01-01 08:37:01,2014-01-01 02:34:38,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399921,2024,LAW,24349438,2024-12-31 23:23:53,NaN,W911,SUBJECT SCREAMING,PCHK,PATROL CHECK,115.0,...,2.366667,0.039444,0.031944,0.071389,2024-12-31 23:25:48,2024-12-31 23:43:46,2024-12-31 23:09:53,0.0,1.0,0
399922,2024,LAW,24349443,2024-12-31 23:28:56,NaN,W911,UNKNOWN PROBLEM,PCHK,PATROL CHECK,645.0,...,6.183333,0.103056,0.179167,0.282222,2024-12-31 23:39:41,2025-01-01 00:03:43,2024-12-31 23:09:53,0.0,1.0,0
399923,2024,LAW,24349460,2024-12-31 23:45:33,NaN,PHONE,LOCATION WANTED SUBJECT,FUP,FOLLOW UP INVESTIGATION,7351.0,...,47.916667,0.798611,2.041944,2.840556,2025-01-01 01:48:04,2025-01-01 04:51:37,2024-12-31 23:09:53,0.0,1.0,0
399924,2024,LAW,24349461,2024-12-31 23:46:01,NaN,PHONE,DISPUTE,PCHK,PATROL CHECK,356.0,...,13.583333,0.226389,0.098889,0.325278,2024-12-31 23:51:57,2025-01-01 00:58:53,2024-12-31 23:09:53,0.0,1.0,0


### Load complete, busy log DataFrame into .csv file named epd_with_capacity.csv¶

In [12]:
epd_with_capacity.to_csv('../data/epd_with_capacity.csv', index=False)